# VISUALIZE OUTPUTS FROM ANALYSIS FUNCTIONS

In [ ]:
# init
import importlib
import tTsTGrpUtils as tsutil
import utils_plots as uplots

test = False # for quick testing

# Data paths
root = "/host/verges/tank/data/daniel/3T7T/z/outputs/"
pth_04a_raw = "04a_dl_maps_08Oct2025-104443.pkl"
pth_04b_parc = "04b_dl_maps_parcel_08Oct2025-110220.pkl"
pth_05a_winComp = "05a_winStudy_18Oct2025-225126.pkl" # z or w scores
pth_05b_winComp_ic = "05b_stats_winStudy_grp_18Oct2025-225832.pkl" # flipped
pth_05c_winD = "05c_stats_winD_08Oct2025-140529.pkl" # Has all data from above maps as well. D score btw px and ctrl z-scores
pth_05d_btwD = "05d_btwD_08Oct2025-153038.pkl" # difference in D scores between studies

# Specifications for dl filtering
key_ft = 'feature' # key for feature in dict
foi = ['thickness', 'T1map'] # features of interest
key_ids = [['ctrl_IDs'],['TLE_R_IDs', 'TLE_L_IDs']]


In [ ]:
# Visualize vertex-wsie feature values compared to control distribution for each participant
importlib.reload(uplots)

reimport_src = False

if 'dl_05b_winComp_ic' not in globals() or reimport_src:
    dl_05b_winComp_ic = tsutil.loadPickle(root + pth_05b_winComp_ic, verbose = True)

dl_05b_winComp_ic = tsutil.filt_dl(dl = dl_05b_winComp_ic, 
                                   key_ft = key_ft, foi = foi, 
                                   key_ids = key_ids)

# NOTE. pass z-scores only. Need changes for w-scores
uplots.raw2Z_vis(dl = dl_05b_winComp_ic, save_path = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs/rawToZ", 
               key_dfs_raw = ['df_maps_parc-glsr_mean', 'df_maps_parc-dk25_mean'],
               marks = True)

In [ ]:
# Visualize parcel wise z-score distributions and corresponding D-scores
importlib.reload(uplots)

save_path = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs/zToD" # will save raw images to ./raw
reimport_src = True
test = False
idx_d = 'd_TLE_ic_ipsiTo-L' # index of cohen's d values in df_d

if 'dl_winD' not in globals() or reimport_src:
    dl_winD = tsutil.loadPickle(root + pth_05c_winD, verbose = True)

dl_winD = tsutil.filt_dl(dl = dl_winD, 
                        key_ft = key_ft, foi = foi, 
                        key_ids = key_ids)

uplots.z2D_vis(dl=dl_winD, save_path = save_path,
                idx_d = idx_d, test = test)

In [ ]:
# Visualize Cohen's D of 3T and 7T studies and their comparisons  
importlib.reload(uplots)

test = False
reimport_src = True

if 'dl_btwD' not in globals or reimport_src:
       dl_btwD = tsutil.loadPickle(root + pth_05d_btwD, verbose = True)

dl_btwD = tsutil.filt_dl(dl = dl_btwD, 
                        key_ft = key_ft, foi = foi, 
                        key_ids = key_ids)

# TODO. Plot these metrics on cortical, hippocampal surfaces; break down by vertical bar plot;
uplots.btwD_vis(dl = dl_btwD, save_path = save_path,
              key_winD = 'df_d_ic', idx_winD = ['d_TLE_ic_ipsiTo-L'], 
              key_btwD = 'comps_df_d_ic', idx_btwD = ['d_TLE_ic_ipsiTo-L_Δd', 'd_TLE_ic_ipsiTo-L_Δd_by3T', 'd_TLE_ic_ipsiTo-L_Δd_by7T'], 
              test = test)

In [ ]:
# % of vertices within each parcel with extreme z-values
importlib.reload(tsutil)
importlib.reload(uplots)

reimport_src = False
thresh = 2.0 # z-threshold

if reimport_src:
    dl_winD = tsutil.loadPickle(root + pth_05c_winD, verbose = True)
    dl_winD = tsutil.filt_dl(dl = dl_winD, 
                            key_ft = key_ft, foi = foi, 
                            key_ids = key_ids)

# A. Compute extreme vertex counts per parcel and save to dfs in each item in dl
dl_vrtxCounts, path = tsutil.get_xtremeVrtx(dl = dl_winD, key_df = 'df_maps_z', thresh = thresh,
                                     lobes = True,
                                     save_path_dfs = "/host/verges/tank/data/daniel/3T7T/z/outputs/dfs/06a_xtremeVrtxPerParc",
                                     save_path_dl_out = "/host/verges/tank/data/daniel/3T7T/z/outputs",
                                     save_name = f'06a_xtremeVrtxPerParc',
                                     test = False)


In [ ]:
importlib.reload(uplots)
# B. Visualize with horizontal bar graphs
save_path = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs/06a_xtrmeVrtxPerParc"
nums = ["mean", "mdn"]
denom = "m_vertices"
grps = ["TLE_ic", "ctrl"]

# TODO. For hippocampal segmentations, replace index with interpretable name
for num in nums:
    for grp in grps:
        print(f"Visualizing {num} extreme vertices per parcel/region for group: {grp}...")
        key_df_ctx = f'xtremeVrtxPerParc_df_maps_z_thresh-2p0_{grp}_lobes_stats'
        key_df_hip = f'xtremeVrtxPerParc_df_maps_z_thresh-2p0_{grp}_stats'

        uplots.percentXtremeVrtxPerParc_vis(dl = dl_vrtxCounts, grp = grp, key_df_ctx = key_df_ctx, key_df_hip = key_df_hip, numerator = num, denominator = denom, 
                                            save_path = save_path, test = test)

# C. Count % of participants with extreme vertices per lobe/parcel/vertex


In [ ]:
dl = tsutil.loadPickle(root + pth_05c_winD)
foi = ["thickness"]
dl = tsutil.filt_dl(dl = dl, 
                            key_ft = key_ft, foi = foi, 
                            key_ids = key_ids)
tsutil.print_dict(dl)
for item in dl:
    tsutil.printItemMetadata(item)
    print(f"\t{len(item['ctrl_IDs'])} controls | {len(item['TLE_R_IDs'])} TLE_R, {len(item['TLE_L_IDs'])} TLE_L")

In [ ]:
# piriform parcellation